In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/assignments/assignment1/'
FOLDERNAME = 'Coursework/Junior/Spring2025/231N/231_project_research/project_code'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [7]:
import torch
import numpy as np
from pointnet_model import MaskedMiniPointNet
from pointnet_layers import *
from data_utils import *
from torch.utils.data import DataLoader
import pointnet_model
import importlib

# paths
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [8]:
# Data
data = np.random.rand(5, 250, 250)
data_list = [data] * 50

dataset = CubeDataset(data_list)

sample = dataset[0]
print(sample.shape)



(5, 250, 250)


### mini-Pointnet Vectors

In [9]:
def custom_collate_fn(batch, mask_percentage=0.6, kernel=12):
    cubes = torch.stack([torch.tensor(cube, dtype=torch.float32) for cube in batch])  # (B, Z, Y, X)
    B, Z, Y, X = cubes.shape

    masks = torch.ones_like(cubes, dtype=torch.float32)
    num_blocks = int(mask_percentage * (Z * Y * X) / (kernel ** 3))

    visible_cubes = []

    for b in range(B):
        for _ in range(num_blocks):
            zi = np.random.randint(0, max(Z - kernel, 1))
            yi = np.random.randint(0, max(Y - kernel, 1))
            xi = np.random.randint(0, max(X - kernel, 1))
            masks[b, zi:zi + kernel, yi:yi + kernel, xi:xi + kernel] = 0

    masked_cubes = cubes * masks

    return cubes, masked_cubes, masks  # all are (B, Z, Y, X)

sparse_train_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=custom_collate_fn,
    num_workers=0,
    pin_memory=False, # ok because we load data on gpu already ?
    drop_last=True
)


cubes, masked_cubes, masks = next(iter(sparse_train_loader))
print(len(cubes))
print(cubes.shape)
print(masked_cubes.shape)
print(masks.shape)

2
torch.Size([2, 5, 250, 250])
torch.Size([2, 5, 250, 250])
torch.Size([2, 5, 250, 250])


In [10]:

# Making mask
cubes, masked_cubes, masks = next(iter(sparse_train_loader))


In [11]:
importlib.reload(pointnet_model)
B, Z, Y, X = cubes.shape # (2, 30, 64, 64)

visible_cubes = []

for cube, mask in zip(cubes, masks):
  visible_tokens = cube[mask == 1]
  visible_cubes.append(visible_tokens)

cubes = cubes.reshape(B, X * Y * Z, 1)  # now (B, 1, N)
masks = masks.reshape(B, 1, X*Y*Z)

# torch.cuda.empty_cache()

mini_pointnet = MaskedMiniPointNet(channels=1, feature_dim=512).to(device)
embeddings = mini_pointnet(cubes.to(device), masks.to(device))
print(embeddings.shape)

torch.Size([2, 512])


In [14]:
from pos_embed_model import LearnedPositionalEncoder

pos_encoder = LearnedPositionalEncoder(num_channels=3, embed_dim=512)
pos_encoder.to(device)

# Method to get voxel positions
# B, Z, Y, X = cubes.shape
positions = torch.stack(torch.meshgrid(
    torch.arange(Z),
    torch.arange(Y),
    torch.arange(X),
    indexing='ij'  # new PyTorch API to match np.meshgrid
), dim=-1).reshape(-1, 3).to(cubes.device)  # shape: (N, 3)

#For now, ONLY getting pos embeddings for unmasked voxels
# Will do again on masked learnable tokens once they exist :P

positions = positions.to(cubes.device).float()
pos_input = positions.unsqueeze(0).expand(B, -1, -1)  # (B, N, 3)
# voxel_positions = []

# for b in range(B):
#     mask_flat = masks[b].reshape(-1)  # (Z*Y*X,)
#     # pos_visible = positions[mask_flat == 1]  # shape: (N_visible, 3)
#     voxel_positions.append(positions)

pos_embeds = pos_encoder(pos_input.to(device)) # (B, voxel_ids, embed_dim)
print(pos_embeds.shape)

torch.Size([2, 312500, 512])


In [ ]:
from transformer_things import *
'''
TODO:
 - make transformer encoder
 - make inputs to the transformer (figure out how to combine embeddings + pos_embeddings)
'''
input_dim = pos_embeds.shape[1]
num_heads = 8
num_layers = 1

learning_rate = 1e-3
weight_decay = 1.e-4
batch_size = 64
model = VisionTransformer()


encoder_layer = VisionTransformer(input_dim=input_dim, num_heads=num_heads)
transformer = TransformerDecoder(decoder_layer, num_layers=num_layers)